In [ ]:
import torchimport torch.nn as nnimport torch.optim as optimdata = [    ("Solve: (2+3) * 4", "20"),    ("Solve: (3+5) * 2", "16"),]class GPTChainOfThought(nn.Module):    def __init__(self, vocab_size, embed_size, num_heads, num_layers, hidden_dim):        super(GPTChainOfThought, self).__init__()        self.embedding = nn.Embedding(vocab_size, embed_size)        self.positional_encoding = nn.Parameter(torch.randn(1, 100, embed_size))        self.transformer = nn.TransformerEncoder(            nn.TransformerEncoderLayer(embed_size, num_heads, hidden_dim),            num_layers        )        self.fc = nn.Linear(embed_size, vocab_size)    def forward(self, x):        x = self.embedding(x) + self.positional_encoding[:, :x.size(1), :]        x = self.transformer(x)        return self.fc(x[:, -1, :])# Example vocabulary and encodingvocab = {word: i for i, word in enumerate("Solve 2 3 4 5 + * 20 16".split())}def encode(sentence):    return torch.tensor([vocab[word] for word in sentence.split() if word in vocab])model = GPTChainOfThought(len(vocab), embed_size=16, num_heads=2, num_layers=2, hidden_dim=64)criterion = nn.CrossEntropyLoss()optimizer = optim.Adam(model.parameters(), lr=0.01)for epoch in range(10):    total_loss = 0    for question, answer in data:        inputs = encode(question).unsqueeze(0)        targets = encode(answer)        optimizer.zero_grad()        output = model(inputs)        loss = criterion(output, targets)        loss.backward()        optimizer.step()        total_loss += loss.item()    print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")test_input = encode("Solve: (2+3) * 4").unsqueeze(0)output = model(test_input)predicted = torch.argmax(output, dim=1)decoded = {idx: word for word, idx in vocab.items()}print(f"Predicted: {[decoded[idx.item()] for idx in predicted]}")